<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# Clustering con kmeans

Crear un modelo de segmentación para la estimación de los tipos de clientes según su hábito de compras, ingresos, edad, etc<br>
v2.0

### Objetivos: 
*   Preprocesar los datos (descarga, lectura, limplieza y filtrado).
*   Conocer como funciona el algoritmo kmeans.
*   Evaluar el resultado el algoritmo kmeans.



**k-means:** Algoritmo de agrupacmiento, que selecciona los centroides de clúster iniciales utilizando un muestreo basado en una distribución de probabilidad empírica de la contribución de los puntos a la inercia genera.

In [ ]:
#Librerias a implementar
import os
import platform
 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Recolectar datos
<div align="center"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle"></div>

### Código de descarga del dataset

In [ ]:
if os.access('Cust_Segmentation.csv', os.F_OK) is False:
    if platform.system() == 'Windows':
        !curl https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/Cust_Segmentation.csv > Cust_Segmentation.csv
    else:
        !wget Mall_Customers.csv https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/Cust_Segmentation.csv

### `Cust_Segmentation.csv`:
El dataset **`Cust_Segmentation.csv`** contiene datos sobre personas, sus hábitos de consumo, edad, etc, los cuales pueden ser utilizados para dirigir una campaña de publicidad o de ventas.<br>
[Dataset source](https://www.kaggle.com/gangliu/custdatasets)
- **CustomerID** --> id del comprador, ejemplo 5
- **Age** --> edad
- **Edu** --> Nivel de educación
- **Years Emplyed** --> Años que lleva trabajando
- **Income** --> ingreso de dinero anual de la persona en miles de dolares
- **Card Debt** --> Débitos de la tarjeta (gastos)
- **Other Debt** --> Otros gastos
- **Defaulted** --> Deudor (0 --> No), (1 --> Si)
- **Adress** --> Dirección
- **DebtIncomeRatio** --> que tan gastador es la persona

In [ ]:
# Lectura del dataset con Pandas (pd) y el método read_cdv
df = pd.read_csv("Cust_Segmentation.csv")

# Muestra las 5 primelas filas
df.head()

In [ ]:
# Se hace una copia para cambiar los nombres de las columnas y no modificar el DataFrame original.
df1 = df.copy()

# De df1 se accede al atributo columns para cambiar los nombres de las columnas del DataFrame
df1.columns = ['id_cliente', 'edad', 'nivel_educacion', 'años_empleado', 'ingreso', 'gastos_tarjeta', 'otros_gastos', 'deudor', 'direccion', 'gastador']

# Procesar datos
<div align="center"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle"></div>

In [ ]:
# inspeccione el dataset, visualice las columnas que posee y sus primeras filas
# Ver que columnas son categóricas y numéricas
df1.head()

In [ ]:
# Los algoritmos de clustering no funcionan bien con las variables categóricas, 
# obtener un nuevo dataset (df2) sin la columna 'id_cliente', 'deudor', 'direccion' 
# La coulmna "Edu" se puede considerar como numérica no categórica, porque
# cuanto más alto su valor mayor nivel de educación tiene la persona, por lo que
# los "números" tiene un significado
df2 = df1.drop(['id_cliente', 'deudor', 'direccion'], axis=1)
df2.head()

In [ ]:
# Realizar una inspeccion del dataset en búsqueda de elementos faltantes
# Una vez descargado el archivo en Colab.
# Leerlo con Pandas y el método read_csv
# Una vez extraida toda la información se almacena en df
# A partir de df y el método describe(), mostrará la descripción estadistica básica del archivo que se guardará en des
# Crear una fila nueva llamada Nan en el DataFrame  des,
# que indica la cantidad de datos tipo Nan que tiene cada columna.
# Para crear una nueva fila, se utilizará el operador loc, donde se indica el nombre
# de la nueva fila y con que valores se completará.
# La información será de los datos faltantes df.isna().sum()
# Crear una fila nueva llamada %Nan en el DataFrame des,
# Esta fila se completará con los porcentajes de Nan encontrados en cada columna.
des = df2.describe()
des.loc['Nan'] = df2.isna().sum()
des.loc['%Nan'] = (df2.isna().mean())*100
des

In [ ]:
# Cantidad de filas y columnas con shape
# En la ubicación 0 corresponde a las filas
print('Datos disponibles para analizar: ', df2.shape[0])

# Explorar datos
<div align="center"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle"></div>

In [ ]:
# Observar todas las relaciones entre todos los features, utilizar pairplot
# sns, alias de Seaborn
sns.pairplot(df2)
plt.show()

A simple vista es muy dificil distingir grupos relacionando solo dos variables. 

Confiaremos en el algoritmo de clustering y utilizaremos todo los features de entrada

# Entrenar modelo
<div align="center"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle"></div>

In [ ]:
# Crear un numpy array "X" con los features de entrada
# Se analizó si era conveniente realizar la normalización de datos, cuando se
# ensayó se obtuvo un peor resultado. Cuando se realiza clustering hay que tener
# cuidado que el proceso de normalización puede afectar la distribución
# de los datos y por lo tanto alterar el resultado (es cuestión de probar)
# from sklearn.preprocessing import StandardScaler
# df3 = df2.copy()
# X = StandardScaler().fit_transform(df3)
X = df2.values

#### Crear un modelo de segmentación con Kmeans
Parámetros
- n_clusters --> (k) número de clusters/grupos (defecto 5)
- init --> método utilizado para determinar donde comienzan los clusters
 - k-means++ --> mecanismo inteligente para determinar el comienzo (defecto)
 - random --> los centros se determinan aleatoriamente
- max_iter --> cantidad de iteración (defecto 300)

La inertia mide qué tan bien K-Means agrupa un conjunto de datos. Se calcula midiendo la distancia entre cada punto de datos y su centroide, elevando al cuadrado esta distancia y sumando estos cuadrados en un grupo.

Un buen modelo es uno con baja inertia.

Fuente: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

La idea es buscar la cantidad de clusters. Para ello, se debe entrenar diferentes modelos de Kmeans.

In [ ]:
# Entrenar diferentes modelos de Kmeans en un rango de cluster (2, 10)
# Conservar el resultado de "inertial" para utilizar como métrica de seleccion
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

def find_best_model(X, max_clusters=10):

    n_clusters = list(range(2, max_clusters+1)) ## --> [2, 3, 4, 5, 6, 7, 8, 9, 10]
    ensayos = len(n_clusters)                   ## --> 9 (Cantidad de elementos en la lista de clusters)

    # Arma un array con 9 elementos todos ceros, 
    # que se sobreescribirán por un lado con los errore 
    # y por el otro con el score
    array_error = np.zeros((ensayos)) ## --> array([0., 0., 0., 0., 0., 0., 0., 0., 0.])
    array_score = np.zeros((ensayos)) ## --> array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

    for i in range(ensayos): ## --> ensayos = 9
        # En cada iteración se entrenará un modelo de Kmeans,
        # varieando la cantidad de cluster a medida que itere
        # 'k-means++': selecciona los centroides de clúster iniciales.
        #  random_state=0, determina la generación de números aleatorios para la inicialización del centroide, al ser cero
        # producirá los mismos resultados en diferentes llamadas.
        # Entrena con fit
        # Calcula el error con kmeans.inertia_, y a su vez se guarda el array_error.
        # Calcula el score con silhouette_score(X, kmeans.labels_), y a su vez se guarda el array_score
        kmeans = KMeans(n_clusters=n_clusters[i], init="k-means++", random_state=0)
        kmeans.fit(X)
        array_error[i] = kmeans.inertia_
        array_score[i] = silhouette_score(X, kmeans.labels_)

    return array_error, array_score, n_clusters

    

In [ ]:
# Invocamos la función find_best_model, pasándole los valores de X
array_error, array_score, n_clusters = find_best_model(X)

In [ ]:
# Dibujar la evolucion del error (inertia) y deteminar el 
# punto de quiebre (elbow point) para deteminar el mejor k
def graficar_punto_codo(array_error, array_score, n_clusters):
  # Crea el espacio para dibujar y del gráfico.
  fig = plt.figure()
  ax = fig.add_subplot()

  # Se gráfica dos líneas en un mismo gráfico.
  # n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]
  # array_error/array_error.max(), divide cada error por el máximo error
  ax.plot(n_clusters, array_error/array_error.max(), label='error') 
  ax.plot(n_clusters, array_score, label='score')
  # Grilla de fondo y color
  ax.grid('dashed')
  # Leyenda
  ax.legend()
  # Muestra el gráfico
  plt.show()

  # Mejor cluster, se obtiene por la ubicación del máximo score.
  # Para ello, en el array que contiene todos los clusters probados n_clusters
  # Accedemos con cortechetes y se le indica el máximo score con argmax() --> array_score.argmax()
  best_cluster = n_clusters[array_score.argmax()]
  print('El mejor resultado se consigue con clusters=', best_cluster)


In [ ]:
graficar_punto_codo(array_error, array_score, n_clusters)

*   No se obtuvo un resultado muy bueno.
*   Al observar el gráfico el punto de codo (elbow) figura en K=4.

In [ ]:
# En este caso, intentaremos eliminar dos columnas más; 'años_empleado', 'edad'
# Y se volverá a llamar a la función  find_best_model para entrenar con las columnas que quedaron.
df4 = df2.drop(['años_empleado', 'edad'], axis=1)
X2 = df4.values

array_error2, array_score2, n_clusters2 = find_best_model(X2)

In [ ]:
graficar_punto_codo(array_error2, array_score2, n_clusters2)

Reduciendo un poco los features se obtuvo un mejor modelo, aún el punto de codo no coincide con k=4, pero se ve claramente que ese el punto indicado.

In [ ]:
# Entrenar el modelo con el mejor K (el punto de codo)
kmeans = KMeans(n_clusters=4, init="k-means++", random_state=0)
kmeans.fit(X2)
labels = kmeans.predict(X2)

In [ ]:
# Agregar una nueva columna con el nombre de segmentación, columna que agrupa a cada cliente por 
# la categoría encontrada en el modelo.
df4['segmentacion'] = labels
df4.head()

# Validar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline5.png" width="1000" align="middle">

In [ ]:
# Calcular la exactitud (accuracy) del modelo
from sklearn.metrics import silhouette_score
silhouette_score (X2, kmeans.labels_)

# Utilizar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline6.png" width="1000" align="middle">

In [ ]:
# Observar todas las relaciones entre todos los features, utilizar pairplot
sns.pairplot(df4, hue='segmentacion', palette='bright')
plt.show()

In [ ]:
# Observar la relación entre el ingreso y gastos_tarjeta
# pero ahora coloreando con "hue" los puntos según su clase de segmentacion
sns.scatterplot(data=df4, x='ingreso', y='gastos_tarjeta', hue='segmentacion', palette='bright')
plt.show()

Se puede ve que no existe una relación lineal entre ingreso y gastos_tarjeta

In [ ]:
# Array de prueba
X_prueba = pd.DataFrame({'nivel_educacion': [1],	'ingreso':[20], 'gastos_tarjeta':[0.1952],	'otros_gastos':[1.123],	'gastador':[8.9]})

# Se utliza el modelo con  kmeans.predict()
# Pasandoles los valores 
labels = kmeans.predict(X_prueba.values)
labels

In [ ]:
print("Los datos ingresados pertenece al cluster:", labels[0])

# Conclusión
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline7.png" width="1000" align="middle">

La conclusión, es que se puede observar en el último gráfico que a mayor ingreso las personas gastan más.

In [ ]:
# Analizar la distribución de los gastos_tarjeta en el dataset, a través de un gráfico de caja
fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot()
sns.boxplot(x=df4['gastos_tarjeta'], ax=ax)
ax.grid('dashed')

En conclusión, se nota que los datos no están balanceados en su totalidad  y que hay un grupo importante de ingresos muy alto. Pero que el modelo pudo agrupar. 